# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

#### Run shared detector on bash
```bash
python3 /home/kiro-ros/Projects/rnb-planning/src/scripts/demo_202107/demo_utils/shared_detector.py
```

#### Check and request ip setting from mobile udp client (robot-side)

#### 0.1.1 Set parameters and create planning scene

In [1]:
import numpy as np

CONNECT_INDY = True
CONNECT_MOBILE = True

CONNECT_TASK_PLANNER = False
VISUALIZE = False
VERBOSE = False
PLANNING_MULTIPROC = True
N_AGENTS = 10
NODE_TRIAL_MAX = N_AGENTS * 2
MAX_SOL_NUM = 5
BASE_COST_CUT = 110

TIMEOUT_MOTION = 0.5
TIMEOUT_FULL = 5

ROS_MASTER_ON_MOBILE = False
# Tool dimensions
TOOL_DIM = [0.24, 0.35]
TOOL_CENTER_OFF = [0, -0.035, 0]
TOOL_THICKNESS = 0.11 + 0.05

INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6

COL_COLOR = (1,1,1,0.2)

USE_SDK = False

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()

from pkg.detector.multiICP.multiICP import *
from pkg.detector.detector_interface import DetectionLevel
from pkg.detector.multiICP.config import *

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *
from demo_utils.area_select import DATASET_DIR, SweepDirections
from demo_utils.demo_config import *
from demo_utils.detection_util import *

from pkg.utils.shared_function import *
clear_channels_on("kiromobilemap")

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7kiro, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})
ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

s_builder = SceneBuilder(None)
SceneBuilder.autostart_roscore = not ROS_MASTER_ON_MOBILE
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)
gscene.set_workspace_boundary(
    -1, 2.5, -0.5, 3, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=CAM_LINK, center=(-0.0785, 0, 0.073))

add_brush(gscene, face_name=TOOL_NAME, tool_link=TIP_LINK,
          thickness=TOOL_THICKNESS, tool_dim=TOOL_DIM,
          col_color=COL_COLOR, center_off=TOOL_CENTER_OFF)

HOME_POSE = crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, 
                      incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

mplan.reset_PRQdict(enable_PRQ=0.5, radii=5e-2)
for tip_dir, SWEEP_AXIS in [
    (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)+"-PRQ.pkl"
    PRQ_PATH = os.path.join(SWEEP_DATA_DIR, filename)
    try:
        Pos_Rotvec_Qlist_dict = load_pickle(PRQ_PATH)
        mplan.register_PRQ(ROBOT_NAME, Pos_Rotvec_Qlist_dict, decimal=2)
        print("Loaded: {}".format(filename))
    except:
        print("File not exist: {}".format(filename))
        continue

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=NODE_TRIAL_MAX)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, \
    Gripper2Tool, PlacePlane, SweepFramer

brush_face = pscene.create_binder(
    bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
    point=(0,0, -gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE), 
    rpy=(0,0,0))

gscene.create_safe(
    gtype=GEOTYPE.BOX, name="floor_box", link_name="base_link",
    dims=(15,15,0.4), center=(0,0,0), rpy=(0,0,0), 
    color=(1, 1, 1, 0.1), display=True, collision=False, fixed=True)

gscene.add_highlight_axis("hl", "base_coord", T=np.identity(4), dims=(0.5,0.1,0.1))

kmb = crob.robot_dict["kmb0"]
indy = crob.robot_dict["indy1"]
mobile_box = gscene.NAME_DICT['kmb0_platform_Box_2']
crob.simulator.set_gscene(gscene)
# indy.ktool.led_on()
# indy.ktool.led_off()
# time.sleep(2)
# indy.ktool.send_degree(-45)

if CONNECT_MOBILE:
    assert np.sum(np.abs(get_xyzw_cur()))>1e-4, "UDP Server not connected"
    
if CONNECT_TASK_PLANNER:
    from demo_proto.DisinfectionOperationServicer import serve_on_thread
    servicer = serve_on_thread()

connection command:
kmb0: True
indy1: True
[MOBILE ROBOT] bind: ('192.168.0.10', 50306)
[MOBILE ROBOT] Start UDP THREAD
goal reach: 0 -> 1 (1639993661.72)
goal reach signal edge up
==== Kiro Tool connected to /dev/ttyUSB0 (115200) ====
[KTOOL] initialize
[KTOOL] enable
[KTOOL] recv [161, 0, 0, 0]
[KTOOL] op_init
[KTOOL] recv [163, 255, 0, 0]
[KTOOL] led_off
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Loaded: indy7kiro-frontX-PRQ.pkl
Loaded: indy7kiro-upZ-PRQ.pkl
Loaded: indy7kiro-downZ-PRQ.pkl


#### 0.1.2 Load environment map

In [3]:
from demo_utils.ros_map_utils import KiroMobileMap
kmm = KiroMobileMap(MOBILE_IP, IP_CUR, CONNECT_MOBILE)
            
VALID_BOX_SCALE = 0.8
VALID_SCORE_CUT = 50
kmb.coster = (lambda Q: 
                  np.max(
                      kmm.get_box_costs(mobile_box, Q, kmm.T_bi, kmm.cost_im, kmm.resolution, 
                                        scale=VALID_BOX_SCALE)))
kmb.cost_cut = VALID_SCORE_CUT
kmb.gscene = gscene

kmm.init_node(timeout=10)

In [4]:
pole_pt_list, pole_res = kmm.update_map(gscene, crob, MOBILE_BASE, timeout=100)
if not CONNECT_MOBILE:
    crob.joint_move_make_sure(kmm.Q_map)
    
if CONNECT_INDY:
    indy.move_joint_s_curve(crob.home_pose[6:], N_div=500)
else:
    indy.joint_move_make_sure(crob.home_pose[6:])

Qcur = crob.get_real_robot_pose()

Connect: Server IP (192.168.0.3)
[WARN] Vertices for mesh should be have center point (0,0,0). Auto adjusting.
Connect: Server IP (192.168.0.3)
Move Tool: -2578.31007809 (1639993676.89)
Connect: Server IP (192.168.0.3)
Move finished!
Connect: Server IP (192.168.0.3)


In [5]:
# pole_list = kmm.add_pixel_poles("obs_pt", gscene, pole_pt_list, pole_res, height=WS_HEIGHT)
# gcheck.ignore_always = pole_list

# gscene.update_markers_all() 

## 1. Prepare scene

In [6]:
HEIGHT_OFFSET = 0.01
TABLE_DIMS = (0.60,1.2, 0.74+HEIGHT_OFFSET)
TABLE_LOC = (-0.1,1.85,TABLE_DIMS[2]/2)
TABLE_ANG = 0
TABLE_MARGIN_I = 0.03
TABLE_MARGIN_O = 0.03
CLEARANCE = 0.002
TABLE_COLOR = (0.8,0.8,0.8,1)
SURF_COLOR = (0,1,0,0.4)
COL_COLOR = (1,1,1,0.2)

gscene.create_safe(GEOTYPE.BOX, "table", link_name="base_link",
                   dims=TABLE_DIMS, center=TABLE_LOC, rpy=Rot2rpy(Rot_axis(3, TABLE_ANG)),
                   color=TABLE_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "table_col", link_name="base_link",
                   dims=np.add(TABLE_DIMS, 
                               (TABLE_MARGIN_O*2, TABLE_MARGIN_O*2, 0)), 
                   center=(0,0,0), rpy=(0,0,0), color=COL_COLOR, 
                   collision=True, parent="table"
                  )
table_surf = gscene.create_safe(GEOTYPE.BOX, "table_surf", link_name="base_link",
                   dims=np.subtract(TABLE_DIMS[:2]+(CLEARANCE,), 
                                    (TABLE_MARGIN_I*2, TABLE_MARGIN_I*2, 0)), 
                   center=(0,0,(TABLE_DIMS[2]+CLEARANCE)/2), 
                   rpy=(0,0,0), color=SURF_COLOR, parent="table",
                   collision=False
                  )

In [7]:
SOFA_DIMS = (0.49,1.0, 0.43)
SOFA_LOC = (0.05,0.7,SOFA_DIMS[2]/2+HEIGHT_OFFSET)
SOFA_SIDE_THIC = 0.2
SOFA_SIDE_HEIT = 0.54
SOFA_ANG = 0

SOFA_MARGIN_O = 0.03
SOFA_MARGIN_I = 0.05
SOFA_UV_DIST = 0.15

SOFA_COLOR = (0.4,0.4,0.4,1)

sofa = gscene.create_safe(GEOTYPE.BOX, "sofa", link_name="base_link",
                   dims=SOFA_DIMS, center=SOFA_LOC, 
                   rpy=Rot2rpy(Rot_axis(3, SOFA_ANG)),
                   color=SOFA_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_col", link_name="base_link",
                   dims=np.add(SOFA_DIMS, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa"
                  )
# sofa_left = gscene.create_safe(GEOTYPE.BOX, "sofa_left", link_name="base_link",
#                    dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
#                    center=(0,(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
#                    rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
#                   )
# gscene.create_safe(GEOTYPE.BOX, "sofa_left_col", link_name="base_link",
#                    dims=np.add(sofa_left.dims, SOFA_MARGIN_O*2), 
#                    center=(0,0,0), rpy=(0,0,0),
#                    color=COL_COLOR, collision=True, parent="sofa_left"
#                   )
# sofa_right = gscene.create_safe(GEOTYPE.BOX, "sofa_right", link_name="base_link",
#                    dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
#                    center=(0,-(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
#                    rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
#                   )
# gscene.create_safe(GEOTYPE.BOX, "sofa_right_col", link_name="base_link",
#                    dims=np.add(sofa_right.dims, SOFA_MARGIN_O*2), 
#                    center=(0,0,0), rpy=(0,0,0),
#                    color=COL_COLOR, collision=True, parent="sofa_right"
#                   )
sofa_back = gscene.create_safe(GEOTYPE.BOX, "sofa_back", link_name="base_link",
#                    dims=(SOFA_SIDE_THIC, SOFA_DIMS[1]+SOFA_SIDE_THIC*2, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   dims=(SOFA_SIDE_THIC, SOFA_DIMS[1], SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(-(SOFA_DIMS[0]+SOFA_SIDE_THIC)/2, 0, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_back_col", link_name="base_link",
                   dims=np.add(sofa_back.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_back"
                  )

sofa_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_surf", link_name="base_link",
                               dims=tuple(np.subtract(SOFA_DIMS[:2], SOFA_MARGIN_I*2))+(CLEARANCE,), 
                               center=(0,0,SOFA_DIMS[2]/2+SOFA_UV_DIST), collision=False, 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa"
                              )

sofa_back_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_back_surf", link_name="base_link",
                               dims=sofa_back.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(SOFA_MARGIN_I*2.5,0,sofa_back.dims[2]/2+SOFA_MARGIN_O+CLEARANCE), 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_back"
                              )

# sofa_left_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_left_surf", link_name="base_link",
#                                dims=sofa_left.dims[:2]+(CLEARANCE,), collision=False, 
#                                center=(0,0,sofa_left.dims[2]/2+SOFA_UV_DIST), 
#                                rpy=(0,0,0), color=SURF_COLOR, parent="sofa_left"
#                               )

# sofa_right_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_right_surf", link_name="base_link",
#                                dims=sofa_right.dims[:2]+(CLEARANCE,), collision=False, 
#                                center=(0,0,sofa_right.dims[2]/2+SOFA_UV_DIST), 
#                                rpy=(0,0,0), color=SURF_COLOR, parent="sofa_right"
#                               )

sofa_front = gscene.create_safe(GEOTYPE.BOX, "sofa_front", link_name="base_link",
                               dims=(sofa.dims[2], sofa.dims[1])+(CLEARANCE,), 
                               center=(sofa.dims[0]/2+SOFA_UV_DIST,0,SOFA_MARGIN_I), collision=False, 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa"
                              )

# sofa_left_front = gscene.create_safe(GEOTYPE.BOX, "sofa_left_front", link_name="base_link",
#                                dims=(sofa_left.dims[2], sofa_left.dims[1])+(CLEARANCE,), 
#                                center=(sofa_left.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_I), collision=False, 
#                                rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_left"
#                               )

# sofa_right_front = gscene.create_safe(GEOTYPE.BOX, "sofa_right_front", link_name="base_link",
#                                dims=(sofa_right.dims[2], sofa_right.dims[1])+(CLEARANCE,), 
#                                center=(sofa_right.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_I), collision=False, 
#                                rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_right"
#                               )

sofa_back_front = gscene.create_safe(GEOTYPE.BOX, "sofa_back_front", link_name="base_link",
#                                dims=(sofa_back.dims[2]-SOFA_DIMS[2]-SOFA_MARGIN_I, sofa_back.dims[1]-SOFA_SIDE_THIC*2-SOFA_MARGIN_I*3)+(CLEARANCE,), 
                               dims=(sofa_back.dims[2]-SOFA_DIMS[2]-SOFA_MARGIN_I, sofa_back.dims[1]-SOFA_MARGIN_I*3)+(CLEARANCE,), 
                               center=(sofa_back.dims[0]/2+SOFA_UV_DIST,0,SOFA_DIMS[2]/2+SOFA_MARGIN_I*1.5), 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, collision=False, parent="sofa_back"
                              )

In [8]:
pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["room_ws"], 
                    pole_pt_list, Qcur, inside=False, margin=0.1)
pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["table_col"], 
                    pole_pt_list, Qcur, margin=0.1)
pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_col"], 
                    pole_pt_list, Qcur, margin=0.1)
# pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_back_col"], 
#                     pole_pt_list, Qcur, margin=0.1)
# pole_pt_list = kmm.remove_poles_by_box(gscene, gscene.NAME_DICT["sofa_right_col"], 
#                     pole_pt_list, Qcur, margin=0.1)

pole_list = kmm.add_pixel_poles("obs_pt", gscene, pole_pt_list, pole_res, height=WS_HEIGHT)
gcheck.ignore_always = pole_list

gscene.update_markers_all() 

### Set UI

In [9]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


### 1.0 Wait task start queue

In [10]:
if CONNECT_TASK_PLANNER:
    while servicer.object_info_running.object_id < 0:
        time.sleep(1)

   Use a production WSGI server instead.
 * Debug mode: off



## 2. Prepare cleaning

In [11]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *
from pkg.utils.code_scraps import get_look_motion

In [12]:
# mplan.reset_log(flag_log=True)
Qcur = crob.get_real_robot_pose()
Qcur[-1] = np.deg2rad(-45)
Qcur[6:] = crob.home_pose[6:]
crob.joint_move_make_sure(Qcur)
Qcur = crob.get_real_robot_pose()
if CONNECT_INDY:
    indy.ktool.led_off()

Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -2578.31007809 (1639993680.19)
Move Tool: -2578.31007809 (1639993680.46)
Connect: Server IP (192.168.0.3)
[KTOOL] led_off


In [13]:
# mplan.reset_log(flag_log=True)
Qcur = crob.get_real_robot_pose()
Qcur[-1] = np.deg2rad(-45)
if not CONNECT_INDY:
    Qcur[6:] = crob.home_pose[6:]
crob.joint_move_make_sure(Qcur)
HOME_POSE_SWEEP = np.copy(Qcur)
# HOME_POSE_SWEEP[6:] = 0
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, 
                           gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    

VEL_LIMS = 0.4
ACC_LIMS = 0.4
SWP_VEL_LIMS = 0.2
SWP_ACC_LIMS = 0.2
RADI_DEG = 1

if CONNECT_INDY:
    indy.QVEL_LEVEL = 1
#     indy.collision_policy = indy_trajectory_client_nosdk.POLICY_NO_COLLISION_DETECTION
    indy.reset()

gtimer = GlobalTimer()
    
mode_switcher=ModeSwitcherLED(pscene, robot_name=ROBOT_NAME, brush_face=brush_face.geometry)

def no_offset(gxter, crob, mplan, robot_name, Qref):
    return Qref, Qref

def cleaning_fun(surface, tip_dir, sweep_dir, tool_dir, Qcur,
                 swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS):
    if sweep_dir.lower() == "z":
        EE_HEIGHT = None
    else:
        T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
        T_brush_e = SE3_inv(T_e_brush)
        EE_HEIGHT = round(surface.get_tf(HOME_DICT)[2,3] + surface.dims[2]/2, 5) \
                        + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
    gtimer.reset(scale=1, timeunit='s', stack=True)
    gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
                           vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
                           swp_vel_lims=swp_vel_lims, swp_acc_lims=swp_acc_lims
                          )

    gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                          timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                          verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

    gxter.get_division_dict(surface, tip_dir, sweep_dir, EE_HEIGHT)
    gxter.init_base_divs(Qcur)
    # gxter.mark_tested(None, None, covereds_all, [])
    snode_schedule_list, Qcur, covereds = gxter.greedy_execute(
        Qcur, tool_dir, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

    gxter.test_clear()
    print(gtimer)
    gscene.clear_virtuals()
    return snode_schedule_list, Qcur

Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -2578.31007809 (1639993681.39)
Move Tool: -2578.31007809 (1639993681.67)
Connect: Server IP (192.168.0.3)


## 3. Table cleaning

In [27]:
snode_schedule_list_table_all = []

In [15]:
snode_schedule_list_table, Qcur = cleaning_fun(surface=table_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
snode_schedule_list_table_all.append(snode_schedule_list_table)

('Height Reference: ', 0.47399999845027924)
Send to mobile: [ 1.147  1.671  1.    -0.003]
             -> [ 0.927  1.683  1.     0.   ]
Distance=0.220487296851 ([-0.22   0.012 -0.007])
goal reach: 1 -> 0 (1639990654.88)
goal reach: 0 -> 1 (1639990675.48)
goal reach signal edge up
End up at=[ 0.926  1.724  3.137] (0.0408 / 0.0408)
[PLAN] Adjust base once. ((1.0269999999999999, -0.16700000000000001, -0.751), (0.0, 0.0, 1.0, 0.0)) / ((1.03, -0.17000000000000001, -0.75), (1.0, 0.0))
[PLAN] Qcur: [ 0.927  1.683  3.142]
[PLAN] Qref: [ 0.927  1.683  3.142]
[PLAN] tar: [ 0.927  1.683  3.142]
Send to mobile: [ 0.926  1.724  1.     0.002]
             -> [ 0.927  1.683  1.     0.   ]
Distance=0.0407588948865 ([ 0.001 -0.041  0.004])
goal reach: 1 -> 0 (1639990676.98)
goal reach: 0 -> 1 (1639990679.38)
goal reach signal edge up
End up at=[ 0.93   1.681  3.14 ] (0.00329 / 0.00329)
[PLAN] Line idc [0, 4]
[PLAN] Try idc (0, 4)
Use 10/12 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal

[PLAN] Line idc [2, 6]
[PLAN] Try idc (2, 6)
Use 10/12 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 3: required answers acquired  (1.1/5.0) ===============================
Goal reached
Goal reached
======================= terminated 2: required answers acquired  (1.2/5.0) ===============================
======================= terminated 5: required answers acquired  (1.2/5.0) ===============================
======================= terminated 6: Stop called from other agent  (1.2/5.0) ===============================
Goal reached
======================= terminated 8: required answers acquired  (1.3/5.0) ===============================
Goal reached
======================= terminated 1: required answers acquired  (1.4/5.0) ===============================
Goal reached
======================= terminated 7: required answers acquired  (1.4/5.0) ===============================
Goal reached
======================= termin

## 4. Sofa cleaning

### 4.1 upside

In [14]:
snode_schedule_list_sofa_all = []

In [15]:
snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

('Height Reference: ', 0.3129999984502792)
Send to mobile: [ 1.176  0.073  1.    -0.01 ]
             -> [ 1.182  0.547  1.     0.   ]
Distance=0.474226874374 ([ 0.006  0.474 -0.02 ])
goal reach: 1 -> 0 (1639993698.72)
goal reach: 0 -> 1 (1639993709.12)
goal reach signal edge up
End up at=[ 1.182  0.546 -3.072] (6.21 / 6.21)
[PLAN] Adjust base once. ((1.1319999999999999, -0.153, -0.58999999999999997), (0.0, 0.0, 1.0, 0.0)) / ((1.1299999999999999, -0.14999999999999999, -0.58999999999999997), (1.0, 0.0))
[PLAN] Qcur: [ 1.182  0.547  3.142]
[PLAN] Qref: [ 1.182  0.547  3.142]
[PLAN] tar: [ 1.182  0.547  3.142]
Send to mobile: [ 1.182  0.546  0.999 -0.035]
             -> [ 1.182  0.547  1.     0.   ]
Distance=0.0691651557211 ([ 0.     0.001 -0.069])
goal reach: 1 -> 0 (1639993710.52)
goal reach: 0 -> 1 (1639993712.62)
goal reach signal edge up
End up at=[ 1.184  0.549  3.118] (0.0239 / 0.0239)
[PLAN] Line idc [0, 3]
[PLAN] Try idc (0, 3)
Use 10/12 agents
Goal reached
Goal reached
Goal rea

======================= terminated 2: required answers acquired  (0.9/5.0) ===============================
Goal reached
======================= terminated 3: required answers acquired  (0.9/5.0) ===============================
Goal reached
Goal reached
======================= terminated 7: required answers acquired  (1.0/5.0) ===============================
======================= terminated 0: required answers acquired  (1.0/5.0) ===============================
Goal reached
======================= terminated 5: required answers acquired  (1.0/5.0) ===============================
Goal reached
======================= terminated 1: required answers acquired  (1.0/5.0) ===============================
Goal reached
Goal reached
======================= terminated 6: required answers acquired  (1.1/5.0) ===============================
======================= terminated 4: required answers acquired  (1.1/5.0) ===============================
======================= terminated 8: Stop called fro

Move Tool: -17.4652227347 (1639993731.19)
Connect: Server IP (192.168.0.3)
Move Tool: -15.6055254377 (1639993731.39)
Connect: Server IP (192.168.0.3)
Move Tool: -12.8688687931 (1639993731.59)
Connect: Server IP (192.168.0.3)
Move Tool: -9.2934199376 (1639993731.79)
Connect: Server IP (192.168.0.3)
Move Tool: -7.52774673092 (1639993731.99)
Connect: Server IP (192.168.0.3)
Move Tool: -5.77179747991 (1639993732.19)
Connect: Server IP (192.168.0.3)
Move Tool: -5.77179747991 (1639993732.39)
Connect: Server IP (192.168.0.3)
Move Tool: -6.64877328126 (1639993732.59)
Connect: Server IP (192.168.0.3)
Move Tool: -8.40914052342 (1639993732.79)
Connect: Server IP (192.168.0.3)
Move Tool: -11.0724755878 (1639993732.99)
Connect: Server IP (192.168.0.3)
Move Tool: -14.686908999 (1639993733.19)
Connect: Server IP (192.168.0.3)
Move Tool: -16.5313995097 (1639993733.4)
Connect: Server IP (192.168.0.3)
Move Tool: -18.4076033941 (1639993733.6)
Connect: Server IP (192.168.0.3)
Move Tool: -18.4076033941 (16

Connect: Server IP (192.168.0.3)
Move Tool: -29.1219206321 (1639993754.86)
Connect: Server IP (192.168.0.3)
Move Tool: -29.1219206321 (1639993755.06)
Connect: Server IP (192.168.0.3)
Move Tool: -29.1219206321 (1639993755.26)
Connect: Server IP (192.168.0.3)
Move Tool: -29.1219206321 (1639993755.46)
Connect: Server IP (192.168.0.3)
Move Tool: -28.1753893093 (1639993755.66)
Connect: Server IP (192.168.0.3)
Move Tool: -27.2527024317 (1639993755.86)
Connect: Server IP (192.168.0.3)
Move Tool: -24.5659911897 (1639993756.07)
Connect: Server IP (192.168.0.3)
Move Tool: -21.1460963342 (1639993756.27)
Connect: Server IP (192.168.0.3)
Move Tool: -18.6808143395 (1639993756.47)
Connect: Server IP (192.168.0.3)
Move Tool: -17.0769947212 (1639993756.67)
Connect: Server IP (192.168.0.3)
Move Tool: -16.2856669944 (1639993756.87)
Connect: Server IP (192.168.0.3)
Move Tool: -16.2856669944 (1639993757.07)
Connect: Server IP (192.168.0.3)
Move Tool: -18.6808143395 (1639993757.27)
Connect: Server IP (192.1

In [17]:
indy.QVEL_LEVEL

1

In [18]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_back_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [19]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_right_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [20]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_left_surf, tip_dir="front", sweep_dir="X", tool_dir=1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [21]:
snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_back_front, tip_dir="up", sweep_dir="Z", tool_dir=1, Qcur=Qcur)
snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

Send to mobile: [ 1.184  0.549  1.     0.012]
             -> [ 1.184  0.31   1.     0.   ]
Distance=0.240075375513 ([-0.    -0.239  0.024])
goal reach: 1 -> 0 (1639993791.92)
goal reach: 0 -> 1 (1639993798.02)
goal reach signal edge up
End up at=[ 1.178  0.309  3.109] (0.0335 / 0.0335)
[PLAN] Adjust base once. ((0.78500000000000003, -0.39000000000000001, 1.2290000000000001), (-0.70699999999999996, 0.0, 0.70699999999999996, 0.0)) / ((0.78000000000000003, -0.39000000000000001, 1.23), (0.70999999999999996, 0.0))
[PLAN] Qcur: [ 1.184  0.31   3.142]
[PLAN] Qref: [ 1.184  0.31   3.142]
[PLAN] tar: [ 1.184  0.31   3.142]
Send to mobile: [ 1.178  0.309  1.     0.016]
             -> [ 1.184  0.31   1.     0.   ]
Distance=0.0334532556956 ([ 0.006  0.001  0.033])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 4.18
[WARN] ROBOT MOTION TIMEOUT
End up at=[ 1.178  0.309  3.109] (0.0335 / 0.0335)
[PLAN] Line idc [0, 3]
[PLAN] Try idc (0, 3)
Use 10/12 agents
======================= terminated 4: node q

Goal reached
======================= terminated 0: required answers acquired  (0.8/5.0) ===============================
======================= terminated 3: required answers acquired  (0.8/5.0) ===============================
Goal reached
Goal reached
======================= terminated 4: required answers acquired  (0.8/5.0) ===============================
======================= terminated 5: required answers acquired  (0.8/5.0) ===============================
======================= terminated 8: Stop called from other agent  (0.8/5.0) ===============================
Goal reached
======================= terminated 6: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 1: required answers acquired  (0.8/5.0) ===============================
Goal reached
======================= terminated 7: required answers acquired  (0.9/5.0) ===============================
========================== FINISHED (0.9 / 5.0 s) =============

======================= terminated 9: Stop called from other agent  (1.1/5.0) ===============================
Goal reached
======================= terminated 8: required answers acquired  (1.1/5.0) ===============================
Goal reached
Goal reached
======================= terminated 7: required answers acquired  (1.1/5.0) ===============================
======================= terminated 5: required answers acquired  (1.1/5.0) ===============================
========================== FINISHED (1.1 / 5.0 s) ==============================]
Goal reached
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -45.0 (1639993816.15)
Connect: Server IP (192.168.0.3)
Move Tool: -42.6262174333 (1639993816.36)
Connect: Server IP (192.168.0.3)
Move Tool: -40.2524348665 (1639993816.56)
Connect: Server IP (192.168.0.3)
Move Tool: -37.8786522998 (1639993816.76)
Connect: Server IP (192.168.0.3)
Move Tool: -

Move Tool: 33.4149838491 (1639993835.21)
Connect: Server IP (192.168.0.3)
Move Tool: 32.267650063 (1639993835.42)
Connect: Server IP (192.168.0.3)
Move Tool: 29.9729824909 (1639993835.62)
Connect: Server IP (192.168.0.3)
Move Tool: 28.8256487049 (1639993835.82)
Connect: Server IP (192.168.0.3)
Move Tool: 27.6783149188 (1639993836.02)
Connect: Server IP (192.168.0.3)
Move Tool: 25.3836473467 (1639993836.22)
Connect: Server IP (192.168.0.3)
Move Tool: 24.2363135606 (1639993836.42)
Connect: Server IP (192.168.0.3)
Move Tool: 21.9416459885 (1639993836.62)
Connect: Server IP (192.168.0.3)
Move Tool: 20.7943122025 (1639993836.82)
Connect: Server IP (192.168.0.3)
Move Tool: 19.6469784164 (1639993837.02)
Connect: Server IP (192.168.0.3)
Move Tool: 17.3523108443 (1639993837.22)
Connect: Server IP (192.168.0.3)
Move Tool: 16.2049770582 (1639993837.42)
Connect: Server IP (192.168.0.3)
Move Tool: 13.9103094861 (1639993837.62)
Connect: Server IP (192.168.0.3)
Move Tool: 12.7629757001 (1639993837.82

Connect: Server IP (192.168.0.3)
Move Tool: 16.1050804337 (1639993858.29)
Connect: Server IP (192.168.0.3)
Move Tool: 14.9885470314 (1639993858.49)
Connect: Server IP (192.168.0.3)
Move Tool: 12.7554802269 (1639993858.69)
Connect: Server IP (192.168.0.3)
Move Tool: 11.6389468247 (1639993858.89)
Connect: Server IP (192.168.0.3)
Move Tool: 10.5224134224 (1639993859.09)
Connect: Server IP (192.168.0.3)
Move Tool: 9.40588002019 (1639993859.29)
Connect: Server IP (192.168.0.3)
Move Tool: 8.28934661794 (1639993859.49)
Connect: Server IP (192.168.0.3)
Move Tool: 8.28934661794 (1639993859.69)
Connect: Server IP (192.168.0.3)
Move Tool: 8.28934661794 (1639993859.88)
Connect: Server IP (192.168.0.3)
LED ON
[KTOOL] led_on
[KTOOL] recv [162, 6, 0, 0]
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: 8.28934604275 (1639993860.9)
Connect: Server IP (192.168.0.3)
Move Tool: 8.71460835756 (1639993861.1)
Connec

Connect: Server IP (192.168.0.3)
Move Tool: 14.0791596748 (1639993881.37)
Connect: Server IP (192.168.0.3)
Move Tool: 13.7677350969 (1639993881.57)
Connect: Server IP (192.168.0.3)
Move Tool: 13.456310519 (1639993881.77)
Connect: Server IP (192.168.0.3)
Move Tool: 13.144885941 (1639993881.97)
Connect: Server IP (192.168.0.3)
Move Tool: 12.8334613631 (1639993882.17)
Connect: Server IP (192.168.0.3)
Move Tool: 12.5220367852 (1639993882.37)
Connect: Server IP (192.168.0.3)
Move Tool: 12.2106122073 (1639993882.57)
Connect: Server IP (192.168.0.3)
Move Tool: 11.8991876293 (1639993882.78)
Connect: Server IP (192.168.0.3)
Move Tool: 11.2763384735 (1639993882.97)
Connect: Server IP (192.168.0.3)
Move Tool: 10.9649138955 (1639993883.18)
Connect: Server IP (192.168.0.3)
Move Tool: 10.6534893176 (1639993883.38)
Connect: Server IP (192.168.0.3)
Move Tool: 10.3420647397 (1639993883.58)
Connect: Server IP (192.168.0.3)
Move Tool: 10.0306401618 (1639993883.78)
Connect: Server IP (192.168.0.3)
Move To

======================= terminated 3: node queue empty 3th  (1.9/5.0) ===============================
======================= terminated 1: node queue empty 2th  (1.9/5.0) ===============================
======================= terminated 2: node queue empty 5th  (2.0/5.0) ===============================
======================= terminated 9: node queue empty 4th  (2.0/5.0) ===============================
======================= terminated 7: node queue empty 6th  (2.0/5.0) ===============================
======================= terminated 4: node queue empty 7th  (2.1/5.0) ===============================
======================= terminated 6: node queue empty 8th  (2.2/5.0) ===============================
======================= terminated 8: node queue empty 10th  (2.3/5.0) ===============================
======================= terminated 5: node queue empty 9th  (2.3/5.0) ===============================
========================== FINISHED (2.3 / 5.0 s) ==============================]

======================= terminated 9: node queue empty 1th  (1.5/5.0) ===============================
======================= terminated 3: node queue empty 3th  (1.5/5.0) ===============================
======================= terminated 2: node queue empty 3th  (1.5/5.0) ===============================
======================= terminated 5: node queue empty 5th  (1.5/5.0) ===============================
======================= terminated 1: node queue empty 6th  (1.5/5.0) ===============================
======================= terminated 0: node queue empty 4th  (1.5/5.0) ===============================
======================= terminated 4: node queue empty 7th  (1.6/5.0) ===============================
======================= terminated 7: node queue empty 8th  (1.6/5.0) ===============================
======================= terminated 8: node queue empty 9th  (1.7/5.0) ===============================
======================= terminated 6: node queue empty 10th  (1.7/5.0) ===========

Connect: Server IP (192.168.0.3)
Move Tool: -23.5963309938 (1639993964.78)
Connect: Server IP (192.168.0.3)
Move Tool: -21.9498949164 (1639993964.98)
Connect: Server IP (192.168.0.3)
Move Tool: -18.6570227616 (1639993965.18)
Connect: Server IP (192.168.0.3)
Move Tool: -17.0105866842 (1639993965.38)
Connect: Server IP (192.168.0.3)
Move Tool: -15.3641506068 (1639993965.58)
Connect: Server IP (192.168.0.3)
Move Tool: -12.071278452 (1639993965.78)
Connect: Server IP (192.168.0.3)
Move Tool: -10.4248423746 (1639993965.98)
Connect: Server IP (192.168.0.3)
Move Tool: -8.77840629722 (1639993966.18)
Connect: Server IP (192.168.0.3)
Move Tool: -5.48553414242 (1639993966.38)
Connect: Server IP (192.168.0.3)
Move Tool: -3.83909806502 (1639993966.58)
Connect: Server IP (192.168.0.3)
Move Tool: -0.54622591022 (1639993966.78)
Connect: Server IP (192.168.0.3)
Move Tool: 1.10021016718 (1639993966.98)
Connect: Server IP (192.168.0.3)
Move Tool: 2.74664624458 (1639993967.19)
Connect: Server IP (192.168.

Move Tool: -11.768526745 (1639993987.86)
Connect: Server IP (192.168.0.3)
Move Tool: -13.5086860941 (1639993988.06)
Connect: Server IP (192.168.0.3)
Move Tool: -16.9890047923 (1639993988.26)
Connect: Server IP (192.168.0.3)
Move Tool: -18.7291641413 (1639993988.46)
Connect: Server IP (192.168.0.3)
Move Tool: -22.2094828395 (1639993988.66)
Connect: Server IP (192.168.0.3)
Move Tool: -23.9496421886 (1639993988.86)
Connect: Server IP (192.168.0.3)
Move Tool: -27.4299608868 (1639993989.06)
Connect: Server IP (192.168.0.3)
Move Tool: -29.1701202358 (1639993989.26)
Connect: Server IP (192.168.0.3)
Move Tool: -30.9102795849 (1639993989.46)
Connect: Server IP (192.168.0.3)
Move Tool: -34.3905982831 (1639993989.66)
Connect: Server IP (192.168.0.3)
Move Tool: -36.1307576322 (1639993989.86)
Connect: Server IP (192.168.0.3)
Move Tool: -39.6110763304 (1639993990.06)
Connect: Server IP (192.168.0.3)
Move Tool: -41.3512356794 (1639993990.26)
Connect: Server IP (192.168.0.3)
Move Tool: -43.0913950285 

In [22]:
indy.QVEL_LEVEL

1

In [23]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_right_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [24]:
snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

Send to mobile: [  1.17400000e+00   6.92000000e-01   1.00000000e+00  -1.00000000e-03]
             -> [ 1.552  0.695  1.     0.   ]
Distance=0.377883404328 ([ 0.378  0.003 -0.002])
goal reach: 1 -> 0 (1639993992.43)
goal reach: 0 -> 1 (1639994002.03)
goal reach signal edge up
End up at=[ 1.565  0.728  3.137] (0.0357 / 0.0357)
[PLAN] Adjust base once. ((0.27500000000000002, -0.0050000000000000001, 1.107), (0.70699999999999996, -0.0, -0.70699999999999996, 0.0)) / ((0.28000000000000003, -0.0, 1.1100000000000001), (0.70999999999999996, 0.0))
[PLAN] Qcur: [ 1.552  0.695  3.142]
[PLAN] Qref: [ 1.552  0.695  3.142]
[PLAN] tar: [ 1.552  0.695  3.142]
Send to mobile: [ 1.565  0.728  1.     0.002]
             -> [ 1.552  0.695  1.     0.   ]
Distance=0.0357052677701 ([-0.013 -0.033  0.005])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 4.46
goal reach: 1 -> 0 (1639994003.53)
goal reach: 0 -> 1 (1639994005.13)
goal reach signal edge up
End up at=[ 1.544  0.694 -3.137] (6.28 / 6.28)
[PLAN] Line id

======================= terminated 1: node queue empty 3th  (1.5/5.0) ===============================
======================= terminated 2: node queue empty 4th  (1.5/5.0) ===============================
======================= terminated 7: node queue empty 2th  (1.5/5.0) ===============================
======================= terminated 3: node queue empty 5th  (1.6/5.0) ===============================
======================= terminated 4: node queue empty 6th  (1.6/5.0) ===============================
======================= terminated 6: node queue empty 7th  (1.6/5.0) ===============================
======================= terminated 8: node queue empty 8th  (1.6/5.0) ===============================
======================= terminated 0: node queue empty 9th  (1.7/5.0) ===============================
======================= terminated 9: node queue empty 10th  (1.7/5.0) ===============================
========================== FINISHED (1.8 / 5.0 s) ==============================]

======================= terminated 4: node queue empty 8th  (3.0/5.0) ===============================
======================= terminated 2: node queue empty 9th  (3.0/5.0) ===============================
======================= terminated 9: node queue empty 10th  (3.0/5.0) ===============================
========================== FINISHED (3.0 / 5.0 s) ==============================]
[PLAN] Try idc (5,)
Use 10/12 agents
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
======================= terminated 3: required answers acquired  (0.9/5.0) ===============================
Goal reached
======================= terminated 4: required answers acquired  (0.9/5.0) ===============================
======================= terminated 0: Stop called from other agent  (1.0/5.0) ===============================
======================= terminated 1: Stop called from other agent  (1.0/5.0) ===============================
Goal reached
Goal reached
=======================

Move Tool: -44.5083193145 (1639994019.9)
Connect: Server IP (192.168.0.3)
Move Tool: -44.0166386291 (1639994020.1)
Connect: Server IP (192.168.0.3)
Move Tool: -43.0332772582 (1639994020.3)
Connect: Server IP (192.168.0.3)
Move Tool: -42.5415965727 (1639994020.5)
Connect: Server IP (192.168.0.3)
Move Tool: -41.5582352018 (1639994020.7)
Connect: Server IP (192.168.0.3)
Move Tool: -41.0665545163 (1639994020.9)
Connect: Server IP (192.168.0.3)
Move Tool: -40.5748738309 (1639994021.1)
Connect: Server IP (192.168.0.3)
Move Tool: -40.0831931454 (1639994021.3)
Connect: Server IP (192.168.0.3)
Move Tool: -39.3319619114 (1639994021.5)
Connect: Server IP (192.168.0.3)
Move Tool: -38.5807306774 (1639994021.7)
Connect: Server IP (192.168.0.3)
Move Tool: -37.8294994434 (1639994021.9)
Connect: Server IP (192.168.0.3)
Move Tool: -37.0782682094 (1639994022.1)
Connect: Server IP (192.168.0.3)
Move Tool: -36.3270369754 (1639994022.3)
Connect: Server IP (192.168.0.3)
Move Tool: -35.5758057414 (1639994022.

Move Tool: -47.6004517674 (1639994042.37)
Connect: Server IP (192.168.0.3)
Move Tool: -47.28204534 (1639994042.57)
Connect: Server IP (192.168.0.3)
Move Tool: -46.9280553719 (1639994042.77)
Connect: Server IP (192.168.0.3)
Move Tool: -46.5387208295 (1639994042.97)
Connect: Server IP (192.168.0.3)
Move Tool: -46.33088698 (1639994043.17)
Connect: Server IP (192.168.0.3)
Move Tool: -45.8890686745 (1639994043.37)
Connect: Server IP (192.168.0.3)
Move Tool: -45.6551518924 (1639994043.57)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4077348761 (1639994043.77)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4077348761 (1639994043.98)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4077348761 (1639994044.18)
Connect: Server IP (192.168.0.3)
Move Tool: -45.6551518924 (1639994044.38)
Connect: Server IP (192.168.0.3)
Move Tool: -45.6551518924 (1639994044.58)
Connect: Server IP (192.168.0.3)
Move Tool: -45.8890686745 (1639994044.78)
Connect: Server IP (192.168.0.3)
Move Tool: -46.1143268644 (16

Move Tool: -47.9823340935 (1639994065.65)
Connect: Server IP (192.168.0.3)
Move Tool: -47.056757169 (1639994065.85)
Connect: Server IP (192.168.0.3)
Move Tool: -46.1311802445 (1639994066.05)
Connect: Server IP (192.168.0.3)
Move Tool: -45.2056033201 (1639994066.25)
Connect: Server IP (192.168.0.3)
Move Tool: -44.2800263956 (1639994066.45)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544494711 (1639994066.65)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544494711 (1639994066.85)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544494711 (1639994067.05)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544494711 (1639994067.24)
Connect: Server IP (192.168.0.3)
LED ON
[KTOOL] led_on
[KTOOL] recv [162, 6, 0, 0]
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544487099 (1639994068.25)
Connect: Server IP (192.168.0.3)
Move Tool: -43.3544487099 (1639994068.45)
Connect: Server IP (192.168

Move Tool: -45.5808566613 (1639994088.53)
Connect: Server IP (192.168.0.3)
Move Tool: -45.5808566613 (1639994088.72)
Connect: Server IP (192.168.0.3)
Move Tool: -45.5546702767 (1639994088.92)
Connect: Server IP (192.168.0.3)
Move Tool: -45.5284838922 (1639994089.12)
Connect: Server IP (192.168.0.3)
Move Tool: -45.5022975077 (1639994089.33)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4761111232 (1639994089.53)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4499247387 (1639994089.73)
Connect: Server IP (192.168.0.3)
Move Tool: -45.4237383542 (1639994089.93)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3975519696 (1639994090.13)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3713655851 (1639994090.33)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3713655851 (1639994090.53)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3451792006 (1639994090.73)
Connect: Server IP (192.168.0.3)
Move Tool: -45.3189928161 (1639994090.93)
Connect: Server IP (192.168.0.3)
Move Tool: -45.2928064316

Move Tool: -5.81772826978 (1639994108.57)
Connect: Server IP (192.168.0.3)
Move Tool: -6.24738611268 (1639994108.78)
Connect: Server IP (192.168.0.3)
Move Tool: -6.66341945848 (1639994108.97)
Connect: Server IP (192.168.0.3)
Move Tool: -7.46729147092 (1639994109.17)
Connect: Server IP (192.168.0.3)
Move Tool: -8.60061207056 (1639994109.38)
Connect: Server IP (192.168.0.3)
Move Tool: -9.64352128135 (1639994109.58)
Connect: Server IP (192.168.0.3)
Move Tool: -10.5920946892 (1639994109.78)
Connect: Server IP (192.168.0.3)
Move Tool: -11.4422956415 (1639994109.98)
Connect: Server IP (192.168.0.3)
Move Tool: -11.9523116548 (1639994110.18)
Connect: Server IP (192.168.0.3)
Move Tool: -12.189771021 (1639994110.38)
Connect: Server IP (192.168.0.3)
Move Tool: -12.189771021 (1639994110.58)
Connect: Server IP (192.168.0.3)
Move Tool: -11.9523116548 (1639994110.78)
Connect: Server IP (192.168.0.3)
Move Tool: -11.170083927 (1639994110.98)
Connect: Server IP (192.168.0.3)
Move Tool: -10.286626276 (16

In [25]:
indy.QVEL_LEVEL

1

In [26]:
indy.traj_vel

1

In [ ]:
# snode_schedule_list_sofa, Qcur = cleaning_fun(surface=sofa_left_front, tip_dir="down", sweep_dir="Z", tool_dir=-1, Qcur=Qcur)
# snode_schedule_list_sofa_all.append(snode_schedule_list_sofa)

In [32]:
indy.ktool.op_init()

[KTOOL] op_init
[KTOOL] recv [162, 6, 0, 0]


False

# Save schedule

In [28]:
save_schedules(snode_schedule_list_table_all, snode_schedule_list_sofa_all)

In [29]:
snode_schedule_list_table_all, snode_schedule_list_sofa_all = load_schedules()

[INFO] data_folder is not specified - select most recent data
schedule loaded from /home/kiro-ros/Projects/rnb-planning/data/demo1222/20211220-185601


In [ ]:
for snode_schedule_list in snode_schedule_list_table_all:
    play_cleaning_schedule(ppline, table_surf, snode_schedule_list, mode_switcher, TOOL_DIM)

In [36]:
for snode_schedule_list in snode_schedule_list_sofa_all:
    play_cleaning_schedule(ppline, sofa_surf, snode_schedule_list, mode_switcher, TOOL_DIM)

Send to mobile: [ 1.713  0.693  1.    -0.003]
             -> [ 1.182  0.547  1.     0.   ]
Distance=0.550928139979 ([-0.531 -0.146 -0.007])
goal reach: 1 -> 0 (1639994495.43)
goal reach: 0 -> 1 (1639994508.03)
goal reach signal edge up
End up at=[ 1.175  0.527 -3.14 ] (6.28 / 6.28)
Connect: Server IP (192.168.0.3)
Move Tool: -2578.31007809 (1639994509.46)
Move Tool: -2578.31007809 (1639994509.74)
Connect: Server IP (192.168.0.3)
move_joint_traj: indy1 pose does not match with trajectory initial state. calling joint_move_make_sure
[  0.  50. -70.   0. -90.   0. -45.]
[   -0.     50.    -70.      0.    -90.     -0.  -2578.3]
Connect: Server IP (192.168.0.3)
Move Tool: -2578.31007809 (1639994510.37)
Move Tool: -2578.31007809 (1639994510.65)
joint_move_make_sure done
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -45.0 (1639994511.48)
Connect: Server IP (192.168.0.3)
Move Tool: -45.0 (1639994511.68)
Connect: Server IP (192.168

Connect: Server IP (192.168.0.3)
LED ON
[KTOOL] led_on
[KTOOL] recv [162, 6, 0, 0]
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -18.5947786082 (1639994532.95)
Connect: Server IP (192.168.0.3)
Move Tool: -17.6660717021 (1639994533.16)
Connect: Server IP (192.168.0.3)
Move Tool: -14.9438596186 (1639994533.36)
Connect: Server IP (192.168.0.3)
Move Tool: -11.4062912851 (1639994533.56)
Connect: Server IP (192.168.0.3)
Move Tool: -8.80417662651 (1639994533.76)
Connect: Server IP (192.168.0.3)
Move Tool: -6.23241942038 (1639994533.96)
Connect: Server IP (192.168.0.3)
Move Tool: -5.38000068639 (1639994534.16)
Connect: Server IP (192.168.0.3)
Move Tool: -5.38000068639 (1639994534.36)
Connect: Server IP (192.168.0.3)
Move Tool: -6.23241942038 (1639994534.56)
Connect: Server IP (192.168.0.3)
Move Tool: -8.80417662651 (1639994534.76)
Connect: Server IP (192.168.0.3)
Move Tool: -12.2824297609 (16399945

Move Tool: -23.3434532593 (1639994555.63)
Connect: Server IP (192.168.0.3)
Move Tool: -24.35170844 (1639994555.83)
Connect: Server IP (192.168.0.3)
Move Tool: -25.3599636207 (1639994556.03)
Connect: Server IP (192.168.0.3)
Move Tool: -26.3682188014 (1639994556.23)
Connect: Server IP (192.168.0.3)
Move Tool: -27.3764739821 (1639994556.43)
Connect: Server IP (192.168.0.3)
Move Tool: -28.3847291628 (1639994556.63)
Connect: Server IP (192.168.0.3)
Move Tool: -29.3929843435 (1639994556.83)
Connect: Server IP (192.168.0.3)
Move Tool: -30.4012395242 (1639994557.03)
Connect: Server IP (192.168.0.3)
Move Tool: -31.4094947049 (1639994557.23)
Connect: Server IP (192.168.0.3)
Move Tool: -32.4177498856 (1639994557.43)
Connect: Server IP (192.168.0.3)
Move Tool: -33.4260050663 (1639994557.64)
Connect: Server IP (192.168.0.3)
Move Tool: -34.434260247 (1639994557.85)


KeyboardInterrupt: 

In [31]:
def print_shcedule_list_info(name, snode_schedule_list_all):
    print("="*11+" "+name+" "+"="*11)
    print("{} face count: {}".format(name, len(snode_schedule_list_all)))
    for i_face, snode_schedule_list_face in enumerate(snode_schedule_list_all):
        print(" face {} location count: {}".format(i_face, len(snode_schedule_list_face)))
        for i_loc, snode_schedule_loc in enumerate(snode_schedule_list_face):
            print("  loc {} snode count: {}".format(i_loc, len(snode_schedule_loc)))
            for i_s, snode in enumerate(snode_schedule_loc):
                if snode.traj is not None:
                    print("  snode {} traj length: {}".format(i_s, len(snode.traj)))
    print("="*(24+len(name)))
    
def print_difference(Q, DQ):
    print("original: {}".format(np.round(brush_face.get_tf_handle(Q, from_link=ROBOT_BASE)[:3,3], 3)))
    print("changed : {}".format(np.round(brush_face.get_tf_handle(Q+DQ, from_link=ROBOT_BASE)[:3,3], 3)))

def print_traj_change(traj, DQ_i, DQ_e):
    print("init")
    print_difference(traj[0], DQ_i)
    print("end")
    print_difference(traj[-1], DQ_e)

def update_traj_start_end(traj, DQ_i, DQ_e):
    len_traj = len(traj)
    traj_add = np.multiply((np.arange(len_traj, dtype=float)/(len_traj-1))[:,np.newaxis],
                           np.subtract(DQ_e, DQ_i)[np.newaxis, :])+np.array(DQ_i)[np.newaxis,:]
    return np.add(traj, traj_add)

def print_traj_ie(traj):
    print("initial: {}".format(np.round(brush_face.get_tf_handle(traj[0], from_link=ROBOT_BASE)[:3,3], 3)))
    print("end    : {}".format(np.round(brush_face.get_tf_handle(traj[-1], from_link=ROBOT_BASE)[:3,3], 3)))
    
def plot_compare_traj(traj1, traj2):
    plt.figure(figsize=(15,4))
    plt.subplot(1,3,1)
    plt.title("original")
    plt.plot(traj1)
    plt.subplot(1,3,2)
    plt.title("changed")
    plt.plot(traj2)
    plt.subplot(1,3,3)
    plt.title("difference")
    plt.plot(traj2-traj1)

In [34]:
print_shcedule_list_info("sofa", snode_schedule_list_sofa_all)

=========== sofa ===========
sofa face count: 3
 face 0 location count: 1
  loc 0 snode count: 10
  snode 1 traj length: 34
  snode 2 traj length: 76
  snode 4 traj length: 11
  snode 5 traj length: 76
  snode 7 traj length: 16
  snode 8 traj length: 76
  snode 9 traj length: 30
 face 1 location count: 2
  loc 0 snode count: 10
  snode 1 traj length: 37
  snode 2 traj length: 56
  snode 4 traj length: 37
  snode 5 traj length: 126
  snode 7 traj length: 14
  snode 8 traj length: 126
  snode 9 traj length: 56
  loc 1 snode count: 4
  snode 1 traj length: 51
  snode 2 traj length: 56
  snode 3 traj length: 53
 face 2 location count: 2
  loc 0 snode count: 10
  snode 1 traj length: 58
  snode 2 traj length: 96
  snode 4 traj length: 16
  snode 5 traj length: 56
  snode 7 traj length: 6
  snode 8 traj length: 56
  snode 9 traj length: 47
  loc 1 snode count: 4
  snode 1 traj length: 45
  snode 2 traj length: 96
  snode 3 traj length: 51


In [35]:
play_cleaning_schedule(ppline, sofa_surf, snode_schedule_list, mode_switcher, TOOL_DIM)

NameError: name 'snode_schedule_list' is not defined

### Finish

In [28]:
if CONNECT_TASK_PLANNER:
    servicer.mark_task_finished()

* 맵 폴 추가 v
* 플래닝 마치고 동작 시 빨간 하이라이트 없애기 v
* 스케줄 세이브 / 로드 구현하기 v
* NoSDK 7DoF 구현하기 v

* no-sdk 버전 mode switcher에 닦기 속도 조절 구현하기
* sdk 버전 제어기 개선 - 아래쪽, 먼 곳 닦을 때 토크 과다?